In [1]:
#Basic imports
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
from os import environ, getcwd
import sys

#Import utils functions
curr_dir = getcwd()
sys.path.append(curr_dir+"/../10.Prediction/utils")
from prediction_general_funcs import test_model_iterative_fixed

sys.path.append(curr_dir+"/utils")
from generate_models_dict_whole_domain import generate_models_dict

from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

### Reading env input

In [2]:
#Reading the ligand input
try:
    ligand = environ['ligand']
except:
    ligand = "sm"
print "ligand = "+ligand
    
#Reading the fold input
try: 
    fold = environ['fold']
except:
    fold = "5"
print "fold = "+fold

#Reading the classifier input
try: 
    classifier_method = environ['classifier']
except:
    classifier_method = "XGB"
print "classifier_method = "+classifier_method

ligand = sm
fold = 5
classifier_method = XGB


### Read data

In [3]:
date = "08.06.18"
folds_num = 5

#Read input and sort by domain
features_table = pd.read_csv(curr_dir+"/domain_features_tables/domain_features_"+fold+".csv", sep="\t", index_col=0)
features_table.sort_index(inplace=True)
labels = pd.read_csv(curr_dir+"/domain_labels/train_domain_labels_"+date+".csv", sep="\t", index_col=0)
labels.sort_index(inplace=True)

print "all samples domains #: "+str(features_table.shape[0])

all samples domains #: 327


In [4]:
#Arrange positives and negatives data
ligand_str = ligand+"_label"
domains_positives = labels[labels[ligand_str] == 1].index
domains_negatvies = labels[labels[ligand_str] == 0].index
ligand_features_positives = features_table.loc[domains_positives,:]
ligand_features_negatives = features_table.loc[domains_negatvies,:]

In [5]:
ligand_features_positives.iloc[2]

rna_max                        0.013946
rna_entropy                    0.793161
rna_thresh_25                  1.000000
rna_thresh_50                  0.955882
rna_thresh_75                  0.647059
rna_zthresh_above_0.5          0.007353
rna_zthresh_above_1            0.007353
rna_zthresh_above_2            0.000000
rna_zthresh_above_3            0.000000
rna_sig_win_num              102.000000
domain_length                136.000000
avg_protein_len              481.875803
avg_phastCons                  0.842090
avg_phyloP                     3.671216
avg_maf                        0.000397
avg_blosum                    -0.218913
avg_hindex                    -0.412116
avg_positve_cnt                1.338235
avg_solvent_acc               54.454751
avg_helix_prob                 0.496762
avg_sheet_prob                 0.113306
avg_turn_prob                  0.391322
max_pfam_C                     0.207215
max_pfam_H                     0.291499
max_pfam_K                     0.781000


In [6]:
no_features = features_table.shape[1]
models_dict = generate_models_dict(ligand, classifier_method, ligand_features_positives, ligand_features_negatives, folds_num, no_features)

### Predict for each ligand seperatelly

In [7]:
#%%time

save_to_file = True

#Initialize dictionary
pred_dict = defaultdict(list)
domain_pred_dict = defaultdict(list)
auc_dict = defaultdict(list)
auprc_dict = defaultdict(list)
domain_auc_mean_dict = defaultdict(list)
domain_auprc_mean_dict = defaultdict(list)
domain_auprc_ratio_mean_dict = defaultdict(list)
domain_auc_dict = defaultdict(list)
domain_auprc_dict = defaultdict(list)
domain_auprc_ratio_dict = defaultdict(list)
prec_prob_dict = defaultdict(list)

model = models_dict[classifier_method][ligand][int(fold)]
(ordered_features, model) = test_model_iterative_fixed(pred_dict, None, auc_dict, auprc_dict, None, None, None, None, None, None, prec_prob_dict, ligand_features_positives, 
                                                        ligand_features_negatives, ligand, model, classifier_method, fold, features=[], whole_domain=True)


pred_df = pd.DataFrame.from_dict(pred_dict)
domain_pred_df = pd.DataFrame.from_dict(domain_pred_dict)
#global matrics dfs
auc_df = pd.DataFrame.from_dict(auc_dict)
auprc_df = pd.DataFrame.from_dict(auprc_dict)
#per domain mean dfs
domain_auc_mean_df = pd.DataFrame.from_dict(domain_auc_mean_dict)
domain_auprc_mean_df = pd.DataFrame.from_dict(domain_auprc_mean_dict)
domain_auprc_ratio_mean_df = pd.DataFrame.from_dict(domain_auprc_ratio_mean_dict)
#per domain dfs
domain_auc_df = pd.DataFrame.from_dict(domain_auc_dict)
domain_auprc_df = pd.DataFrame.from_dict(domain_auprc_dict)
domain_auprc_ratio_df= pd.DataFrame.from_dict(domain_auprc_ratio_dict)

#Save to file
if (save_to_file):
    pred_df.to_csv(curr_dir+"/pred_results/per_fold/"+ligand+"_"+classifier_method+"_fold"+fold+"_"+str(folds_num)+"w.csv", sep='\t')
    auc_df.to_csv(curr_dir+"/pred_results/per_fold/"+ligand+"_"+classifier_method+"_fold"+fold+"_"+str(folds_num)+"w_auc.csv", sep='\t')
    auprc_df.to_csv(curr_dir+"/pred_results/per_fold/"+ligand+"_"+classifier_method+"_fold"+fold+"_"+str(folds_num)+"w_auprc.csv", sep='\t')
        
print "Finished ligand "+ligand

fold #: 5
AUC = 0.7373949579831932
AUPRC = 0.7410338037731106
AU prec prob = 0.24304999320208576
Finished sm XGB fold: 5
Finished ligand sm
